# AI Powered HR Assistant for Nestle Policy 

### 1. Import Important Packages 

In [57]:
import os
import openai
from dotenv import load_dotenv

from langchain.document_loaders import PyPDFLoader

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI

from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

### 2. Setup OpenAI API calling Environment

In [58]:
# Setup open api Key
# Load environment variables from .env file if present
load_dotenv()

# Get the OpenAI API key from environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

print("OpenAI API key loaded successfully.")

OpenAI API key loaded successfully.


### 3. Load and process Nestle Policy data 

In [59]:
pdf_path = './Dataset/the_nestle_hr_policy_pdf_2012.pdf'
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"Loaded {len(documents)} documents from the PDF.")
print(f"First page preview: {documents[0].page_content[:2000]}...")

Loaded 8 documents from the PDF.
First page preview: Policy
Mandatory
September  2012
The Nestlé  
Human Resources Policy...


### 4. Vector Store initialization and Embeddings generation --

In [60]:
# Intialize OpenAI Embeddings
embeddings = OpenAIEmbeddings()

In [61]:
# Create or load the Chroma vector store using Chroma
vector_store = Chroma.from_documents(
    documents, 
    embeddings, 
    persist_directory="./chroma_db"
    )

In [62]:
print("Vector store created and embeddings generated.")
print(f"Number of documents in vector store: {vector_store._collection.count()}")

Vector store created and embeddings generated.
Number of documents in vector store: 32


### 5. Intialize LLM from OpenAI

In [ ]:
# Initialize ChatOpenAI model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=500)

In [64]:
print("llm ChatOpenAI model loaded")

llm ChatOpenAI model loaded


In [65]:
# Create a retrieval-based QA chain
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [75]:
print("Retrieval-based QA chain created successfully.")
print(f"Retriever configured to return top {retriever.search_kwargs['k']} similar documents.")
print(f"Retriever search type: {retriever.search_type}")
print(f"Retriever details: {retriever}")
print(f"Vector store details: {vector_store}")
print(f"Retriever : {retriever.get_relevant_documents}")

Retrieval-based QA chain created successfully.
Retriever configured to return top 3 similar documents.
Retriever search type: similarity
Retriever details: tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7d0458ec28a0> search_kwargs={'k': 3}
Vector store details: <langchain_chroma.vectorstores.Chroma object at 0x7d0458ec28a0>
Retriever : <bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7d0458ec28a0>, search_kwargs={'k': 3})>


### 6. Create PROMPT Template for Custom Prompt for Chat Assistant Model for NESTLE ~

In [67]:
template = """

Your are Helpful AI assistant for NESTLE HR policies. Your role is to help Employee to provide them accurate information in human understandable format. 
If you can not find the answer from the provided context, simply state that "I don't have that information". Do not make up an answer.

Use the following context to answer the question - 
Context = {context}

Question = {question}

HR Assistant Reponse:
"""



In [68]:
PROMPT = PromptTemplate(
    template=template, 
    input_variables=["context", "question"]
)

In [69]:
# Create Retrieval QA Chain

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

### 7. Verify & Test HR Assistant Response ~

In [70]:
print("Test HR Assistant ~")
answer = qa_chain({"query": "What is Nestlé's policy on employee training and development?"})
print("\nAnswer:")

Test HR Assistant ~

Answer:
